In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, sum as spark_sum, count, when, isnan, isnull, length, regexp_extract
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.appName("CSVAnalysis").getOrCreate()

# Raw CSV content as a single row
csv_content = "1,2,3,4,5,ABC,XYZ,6,7,ABC,DEFO, 7"
#df = spark.createDataFrame([(csv_content,)], ["raw_data"])
df = spark.createDataFrame([(csv_content)], ["raw_data"])

# Split into individual values
df_values = df.select(explode(split(col("raw_data"), ",")).alias("value"))
df_values.show(truncate=False)


In [0]:
from pyspark.sql import functions as F

In [0]:
df_cast = df_values.withColumn(
    "int_value",
    F.expr("try_cast(value as int)")
)
display(df_cast)

In [0]:
# --------------------
# DF 1: Sum of integers
# --------------------
num_df = df_cast.select("int_value").filter(col("int_value").isNotNull()).agg(F.sum("int_value").alias("sum"))

display(num_df)

In [0]:
# --------------------
# DF 2: Count of each word
# --------------------
word_count_df = df_cast.filter(col("int_value").isNull()) \
                       .select("value") \
                       .groupBy("value") \
                       .count()

# Show results
word_count_df.show()
